In [1]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from xgboost import DMatrix


### Input Data - Cleaned Dataset All

In [2]:

# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

df_All = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Merge Data\merged_data.csv')

print(df_All.head())


                  Date  425_pct_Al     Al2O3   M_Value  C4_pct_Eth  \
0  2023-01-01 03:00:00    6.159550  10.76240  3.901390    2.077530   
1  2023-01-01 09:00:00    6.159750  10.76285  3.893505    2.123750   
2  2023-01-01 10:00:00    6.159985  10.76340  3.884305    2.177670   
3  2023-01-01 18:00:00    6.160285  10.76415  3.843085    2.246995   
4  2023-01-01 23:00:00    6.160715  10.76520  3.772120    2.347140   

   C4_pct_H2O  HydWtr_pct_Ammonia  C4_pct_Hex  HydWtr_Na2O   Butanol  ...  \
0    21.50120            1.229850    0.387521     3.528950  3.268330  ...   
1    21.55375            1.212805    0.399834     3.239975  3.569165  ...   
2    21.61510            1.190910    0.414198     2.880790  3.569165  ...   
3    21.69400            1.147270    0.432667     2.248915  3.674583  ...   
4    21.80795            1.084235    0.459344     1.336207  3.675000  ...   

    FC55576  FFC55553  FFC55555    LC55557    LC90366    LC90368   PI55020  \
0  3.615110  0.900075  0.768297  66.29

In [3]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [4]:
# # # List of columns to exclude from the XGboost feature selection results
# exclude_columns = [
#                     'TI40050',
#                     'TC55555',
#                     'LC55568',
#                     'LC55557',
#                     'Al2O3',
#                     'LC55553',
#                     'M_Value',
#                     'FFC55553',
#                     'DI55580',
#                     'Al2O3',
#                     'C4_pct_Hex',
#                     'LC52572',

# #                  'Date',
# #                  'C4_pct_Hex', 'HydWtr_Na2O',
# #                  'TC55555'  
#                    ]

# # # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [5]:
df_All.columns

Index(['Date', '425_pct_Al', 'Al2O3', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'Butanol', 'Decanol',
       'Ethanol', 'Hexanol', 'Octanol', 'DI55102', 'DI55152', 'DI55580',
       'FC55003', 'FC55009', 'FC55552', 'FC55569', 'FC55576', 'FFC55553',
       'FFC55555', 'LC55557', 'LC90366', 'LC90368', 'PI55020', 'TC55552',
       'TI55021', 'Year'],
      dtype='object')

In [6]:
# df_All = df_All[df_All['Date'] > '2020-06-15 00:00:00']

In [7]:
# df_All.drop('Date', axis=1, inplace=True)




In [8]:
print(df_All.describe())

        425_pct_Al        Al2O3      M_Value   C4_pct_Eth   C4_pct_H2O  \
count  4863.000000  4863.000000  4863.000000  4863.000000  4863.000000   
mean      6.222890    11.538018     3.633789     1.802102    18.017469   
std       0.266864     0.423541     0.097989     0.393184     1.441296   
min       5.020410    10.762400     3.340825     0.565000    13.995000   
25%       6.124507    11.198225     3.567775     1.466838    17.014225   
50%       6.266860    11.561850     3.626065     1.769575    17.800200   
75%       6.378610    11.939825     3.706710     2.014328    18.745375   
max       6.850010    12.201400     3.901390     3.045115    22.718500   

       HydWtr_pct_Ammonia   C4_pct_Hex  HydWtr_Na2O      Butanol      Decanol  \
count         4863.000000  4863.000000  4863.000000  4863.000000  4863.000000   
mean             0.948256     0.378261     1.689514    13.928301     2.731687   
std              0.115171     0.112879     1.725531    17.315615     1.491233   
min      

In [9]:
# # Assuming your data is in filtered_df and you want to predict 'target_column_name'
# df = df_All.drop(['Butanol', 'FC55009', 'PI55004', 'PI55020','TI55021'], axis=1)  # Replace 'target_column_name' with your target column's name
# # y = df_All['Decanol']

In [10]:
# # Keep only rows with specified IDs
# feature_names = [
# '425_pct_Al',
# 'C4_pct_Eth',
# 'C4_pct_H2O',
# 'HydWtr_pct_Ammonia',
# 'HydWtr_Na2O',
# 'DI55102',
# 'DI55152',
# 'FC55003',
# 'FC55009',
# 'FC55552',
# 'FC55569',
# 'FC55576',
# 'FFC55555',
# 'LC90366',
# 'LC90368',
# 'PI55004',
# 'PI55020',
# 'TC55552',
# 'TI55021'
# ]
    

In [11]:
import pickle

# Load the trained XGBoost models
with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Updated Model\C10_xgb_model.sav', 'rb') as model_file:
    c10_model = pickle.load(model_file)
with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C4\5 Preprocessing - Slecected Features\Updated Model\C4_xgb_model.sav', 'rb') as model_file:
    c4_model = pickle.load(model_file)
with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model STEAM - FC55569\5 Preprocessing - Slecected Features\Updated Model\STEAM_xgb_model.sav', 'rb') as model_file:
    steam_model = pickle.load(model_file)

# Initialize the new columns in the original df
df_All['FFC55555_adj'] = df_All['FFC55555']
df_All['Decanol_adj'] = None  # Using None as a placeholder
df_All['Butanol_adj'] = None  # Using None as a placeholder
df_All['FC55569_adj'] = None  # Using None as a placeholder

def predict_value(model, row, feature_names):
    # Ensure 'row' is prepared correctly for your model
    data_for_prediction = row[feature_names].values.reshape(1, -1)
    prediction = model.predict(data_for_prediction)[0]
    return prediction

# Assume 'decanol_feature_names' and 'butanol_feature_names' are defined
# You should define these lists based on the features your models expect
decanol_feature_names = ['425_pct_Al', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'FC55003',
       'FC55009', 'FC55552', 'FC55569', 'FFC55553', 'FFC55555', 'LC90366',
       'LC90368', 'PI55020', 'TC55552', 'TI55021']  
butanol_feature_names = ['425_pct_Al', 'C4_pct_Eth', 'C4_pct_H2O', 'HydWtr_pct_Ammonia',
       'C4_pct_Hex', 'HydWtr_Na2O', 'DI55102', 'DI55152', 'FC55552',
       'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'LC55557', 'LC90366',
       'LC90368', 'TC55552']
steam_feature_names = ['425_pct_Al', 'Al2O3', 'C4_pct_Eth', 'C4_pct_H2O', 'C4_pct_Hex',
       'DI55102', 'DI55152', 'DI55580', 'FC55003', 'FC55576',
       'FFC55555', 'LC90366', 'PI55020']

# Your prediction and adjustment loop
for index, row in df_All.iterrows():
    # Adjust row for prediction if necessary
    row_for_prediction = row.copy()
    row_for_prediction['FFC55555'] = row_for_prediction['FFC55555_adj']
    
    # Process record through model C10 for Decanol
    df_All.at[index, 'Decanol_adj'] = predict_value(c10_model, row_for_prediction, decanol_feature_names)
    
    # Process record through model C4 for Butanol
    df_All.at[index, 'Butanol_adj'] = predict_value(c4_model, row_for_prediction, butanol_feature_names)

    # Process record through Optimization Logic
    while True:
        decanol = df_All.at[index, 'Decanol_adj']
        butanol = df_All.at[index, 'Butanol_adj']
        ffc_adjusted = df_All.at[index, 'FFC55555_adj']

        if decanol < 0.01:
            if ffc_adjusted >= 0.68: # Can be adjusted to the lowest rC4 desired (desired -0.01)
                df_All.at[index, 'FFC55555_adj'] -= 0.01
            else:
                break  # Next Record
        else:
            while decanol >= 0.01:
                if df_All.at[index, 'FFC55555'] >= ffc_adjusted:
                    break  # Next Record
                else:
                    if ffc_adjusted <= 1.19: # Can be adjusted to the highest rC4 desired
                        df_All.at[index, 'FFC55555_adj'] += 0.01
                    else:
                        break
            break

        # Update predictions after adjustment
        row_for_prediction['FFC55555'] = df_All.at[index, 'FFC55555_adj']
        df_All.at[index, 'Decanol_adj'] = predict_value(c10_model, row_for_prediction, decanol_feature_names)
        

    # Update the FC55569_adj column with the output from the steam model
    df_All.at[index, 'Butanol_adj'] = predict_value(c4_model, row_for_prediction, butanol_feature_names)
    df_All.at[index, 'FC55569_adj'] = predict_value(steam_model, row_for_prediction, steam_feature_names)

In [12]:
df_All.head()

,Date,425_pct_Al,Al2O3,M_Value,C4_pct_Eth,C4_pct_H2O,HydWtr_pct_Ammonia,C4_pct_Hex,HydWtr_Na2O,Butanol,Decanol,Ethanol,Hexanol,Octanol,DI55102,DI55152,DI55580,FC55003,FC55009,FC55552,FC55569,FC55576,FFC55553,FFC55555,LC55557,LC90366,LC90368,PI55020,TC55552,TI55021,Year,FFC55555_adj,Decanol_adj,Butanol_adj,FC55569_adj
0,2023-01-01 03:00:00,6.159550,10.76240,3.901390,2.077530,21.50120,1.229850,0.387521,3.528950,3.268330,1.755000,12.803300,0.955000,1.131670,0.794854,1.036050,1.025130,6004.100000,1548.140,37103.500000,6348.740,3.615110,0.900075,0.768297,66.293000,55.829200,55.811500,-1.065310,169.012000,223.161000,2023,0.768297,1.924359,27.380466,6798.42334
1,2023-01-01 09:00:00,6.159750,10.76285,3.893505,2.123750,21.55375,1.212805,0.399834,3.239975,3.569165,1.614165,13.817500,0.960834,1.052501,0.794854,1.036240,1.025323,5205.556667,1186.100,37025.766667,6346.200,5.432285,0.913374,0.770656,66.084333,55.331633,55.368433,-1.055726,169.252333,222.737333,2023,0.770656,2.869824,24.930531,6792.041504
2,2023-01-01 10:00:00,6.159985,10.76340,3.884305,2.177670,21.61510,1.190910,0.414198,2.880790,3.569165,1.614165,13.817500,0.960834,1.052501,0.794854,1.036240,1.025323,5205.556667,1186.100,37025.766667,6346.200,5.432285,0.913374,0.770656,66.084333,55.331633,55.368433,-1.055726,169.252333,222.737333,2023,0.770656,2.906157,23.955406,6799.662109
3,2023-01-01 18:00:00,6.160285,10.76415,3.843085,2.246995,21.69400,1.147270,0.432667,2.248915,3.674583,1.750417,14.145400,1.007082,1.129586,0.794854,1.036412,1.025825,5154.950000,1256.780,37021.525000,6349.200,8.516539,0.912495,0.770907,66.056000,55.008975,55.024700,-1.105172,169.469000,222.352500,2023,0.770907,2.810742,23.461773,6823.70166
4,2023-01-01 23:00:00,6.160715,10.76520,3.772120,2.347140,21.80795,1.084235,0.459344,1.336207,3.675000,1.855417,13.964575,1.055832,1.200418,0.794854,1.036742,1.026116,5322.356000,1329.426,37012.960000,6348.614,7.042021,0.911921,0.770362,66.066580,54.647740,54.650880,-1.195370,169.571000,222.047200,2023,0.770362,2.337304,20.526184,6848.080078


In [13]:

df_steam = df_All[['Date', 'FC55569', 'FC55569_adj']]

df_steam['df_steam_diff'] = df_steam['FC55569'] - df_steam['FC55569_adj']

df_steam

C:\Users\austinsh\AppData\Local\Temp\ipykernel_1472\938155909.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam['df_steam_diff'] = df_steam['FC55569'] - df_steam['FC55569_adj']


,Date,FC55569,FC55569_adj,df_steam_diff
0,2023-01-01 03:00:00,6348.74000,6798.42334,-449.68334
1,2023-01-01 09:00:00,6346.20000,6792.041504,-445.841504
2,2023-01-01 10:00:00,6346.20000,6799.662109,-453.462109
3,2023-01-01 18:00:00,6349.20000,6823.70166,-474.50166
4,2023-01-01 23:00:00,6348.61400,6848.080078,-499.466078
...,...,...,...,...
4858,2023-12-31 18:00:00,6667.34875,6499.240234,168.108516
4859,2023-12-31 19:00:00,6671.48500,6509.672852,161.812148
4860,2023-12-31 20:00:00,6674.11500,6522.296875,151.818125
4861,2023-12-31 21:00:00,6674.19000,6538.788086,135.401914


In [14]:
# Sum each column
column_sums = df_steam.sum()

# Convert the sums into a DataFrame
sum_row = pd.DataFrame(column_sums).transpose()

# Add a label to the sum row
sum_row['Date'] = 'Sum'

# Append the sum row to the original DataFrame
df_steam_with_sum = pd.concat([df_steam, sum_row], ignore_index=True)

# Display the result
df_steam_with_sum


,Date,FC55569,FC55569_adj,df_steam_diff
0,2023-01-01 03:00:00,6348.74,6798.42334,-449.68334
1,2023-01-01 09:00:00,6346.2,6792.041504,-445.841504
2,2023-01-01 10:00:00,6346.2,6799.662109,-453.462109
3,2023-01-01 18:00:00,6349.2,6823.70166,-474.50166
4,2023-01-01 23:00:00,6348.614,6848.080078,-499.466078
...,...,...,...,...
4859,2023-12-31 19:00:00,6671.485,6509.672852,161.812148
4860,2023-12-31 20:00:00,6674.115,6522.296875,151.818125
4861,2023-12-31 21:00:00,6674.19,6538.788086,135.401914
4862,2023-12-31 22:00:00,6669.56125,6543.097168,126.464082


That might be more steam, but How much alcohol is saved?????????

In [15]:
df_C10 = df_All[['Date', 'Decanol', 'Decanol_adj']]

df_C10['Decanol_diff'] = df_C10['Decanol'] - df_C10['Decanol_adj']

df_C10

C:\Users\austinsh\AppData\Local\Temp\ipykernel_1472\928042281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_C10['Decanol_diff'] = df_C10['Decanol'] - df_C10['Decanol_adj']


,Date,Decanol,Decanol_adj,Decanol_diff
0,2023-01-01 03:00:00,1.755000,1.924359,-0.169359
1,2023-01-01 09:00:00,1.614165,2.869824,-1.255659
2,2023-01-01 10:00:00,1.614165,2.906157,-1.291992
3,2023-01-01 18:00:00,1.750417,2.810742,-1.060324
4,2023-01-01 23:00:00,1.855417,2.337304,-0.481887
...,...,...,...,...
4858,2023-12-31 18:00:00,1.449582,2.884599,-1.435016
4859,2023-12-31 19:00:00,1.515000,2.826717,-1.311717
4860,2023-12-31 20:00:00,1.518332,2.883872,-1.36554
4861,2023-12-31 21:00:00,1.521667,2.909295,-1.387627


In [16]:
# Sum each column
column_sums = df_C10.sum()

# Convert the sums into a DataFrame
sum_row = pd.DataFrame(column_sums).transpose()

# Add a label to the sum row
sum_row['Date'] = 'Sum'

# Append the sum row to the original DataFrame
df_C10_with_sum = pd.concat([df_C10, sum_row], ignore_index=True)

# Display the result
df_C10_with_sum


,Date,Decanol,Decanol_adj,Decanol_diff
0,2023-01-01 03:00:00,1.755,1.924359,-0.169359
1,2023-01-01 09:00:00,1.614165,2.869824,-1.255659
2,2023-01-01 10:00:00,1.614165,2.906157,-1.291992
3,2023-01-01 18:00:00,1.750417,2.810742,-1.060324
4,2023-01-01 23:00:00,1.855417,2.337304,-0.481887
...,...,...,...,...
4859,2023-12-31 19:00:00,1.515,2.826717,-1.311717
4860,2023-12-31 20:00:00,1.518332,2.883872,-1.36554
4861,2023-12-31 21:00:00,1.521667,2.909295,-1.387627
4862,2023-12-31 22:00:00,1.525,2.89101,-1.36601


In [17]:
df_C4 = df_All[['Date', 'Butanol', 'Butanol_adj']]

df_C4['Butanol_diff'] = df_C4['Butanol'] - df_C4['Butanol_adj']

df_C4

C:\Users\austinsh\AppData\Local\Temp\ipykernel_1472\1892453332.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_C4['Butanol_diff'] = df_C4['Butanol'] - df_C4['Butanol_adj']


,Date,Butanol,Butanol_adj,Butanol_diff
0,2023-01-01 03:00:00,3.268330,27.380466,-24.112136
1,2023-01-01 09:00:00,3.569165,24.930531,-21.361366
2,2023-01-01 10:00:00,3.569165,23.955406,-20.386241
3,2023-01-01 18:00:00,3.674583,23.461773,-19.78719
4,2023-01-01 23:00:00,3.675000,20.526184,-16.851184
...,...,...,...,...
4858,2023-12-31 18:00:00,22.483750,18.620758,3.862992
4859,2023-12-31 19:00:00,21.815000,18.697474,3.117526
4860,2023-12-31 20:00:00,20.665000,18.526514,2.138486
4861,2023-12-31 21:00:00,19.515000,18.539173,0.975827


In [18]:
# Sum each column
column_sums = df_C4.sum()

# Convert the sums into a DataFrame
sum_row = pd.DataFrame(column_sums).transpose()

# Add a label to the sum row
sum_row['Date'] = 'Sum'

# Append the sum row to the original DataFrame
df_C4_with_sum = pd.concat([df_C4, sum_row], ignore_index=True)

# Display the result
df_C4_with_sum


,Date,Butanol,Butanol_adj,Butanol_diff
0,2023-01-01 03:00:00,3.26833,27.380466,-24.112136
1,2023-01-01 09:00:00,3.569165,24.930531,-21.361366
2,2023-01-01 10:00:00,3.569165,23.955406,-20.386241
3,2023-01-01 18:00:00,3.674583,23.461773,-19.78719
4,2023-01-01 23:00:00,3.675,20.526184,-16.851184
...,...,...,...,...
4859,2023-12-31 19:00:00,21.815,18.697474,3.117526
4860,2023-12-31 20:00:00,20.665,18.526514,2.138486
4861,2023-12-31 21:00:00,19.515,18.539173,0.975827
4862,2023-12-31 22:00:00,18.365,19.18721,-0.82221
